# Parameter search with optuna - BASELINE

## Importing section

In [1]:
import optuna

import numpy as np

import torch
from sklearn.metrics import r2_score
from torch.utils.data import random_split, DataLoader

from etnn import TreeNode
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.routines.run_config import choice_dataset, choice_trainloader, choice_loss, choice_optim
from etnn.tools.training import train_epoch, eval_epoch
from etnn.tools.training_tools import ConfigStore, seeding_all

## Definition of objective function for ETNN

In [2]:
def objective(trial):
    # init default config
    config = ConfigStore(
        in_dim=15,
        hidden_dim=0, #trial.suggest_int("hidden_dim", 16, 512, step=16),
        out_dim=1,
        k=0, #trial.suggest_int("k", 1, 5),
        dataset=-1,
        ds_size=10_000,
        num_gondolas=10,
        num_part_pg=5,
        loss_name='mse',
        optimizer_name='adam',
        num_max_epochs=30, # real: 100
        learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        batch_size=1024,
        early_stop_tol=5,
        use_equal_batcher=trial.suggest_categorical("batcher", [True, False]),
        seed=420,
        label_type=label,
        final_label_factor=1/1000
    )

    # loading dataset
    dataset, df_index = choice_dataset(config, dataset_path)
    # splitting off test dataset
    generator = torch.Generator().manual_seed(config.seed)
    train_ds, val_ds, _ = random_split(
        dataset,
        [1 - test_perc - val_perc, val_perc, test_perc],
        generator=generator
    )

    # loaders
    train_loader = choice_trainloader(config, df_index, train_ds)
    val_loader = DataLoader(val_ds, batch_size=4 * config.batch_size, shuffle=False)

    # build tree
    tree_structure = TreeNode(
        node_type="C",
        children=[
            TreeNode("P", [TreeNode("E", config.num_part_pg)])
            for _ in range(config.num_gondolas)
        ]
    )

    # define device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # set seed for reproducability
    seeding_all(config.seed)

    # define model
    layer_list = [torch.nn.Flatten()]
    features = config.in_dim * config.num_gondolas * config.num_part_pg

    # for each layer create a linear layer and relu (except last one)
    for i in range(trial.suggest_int("n_layers", 1, 5)-1):
        # determine new feature dimension
        new_features = trial.suggest_int(f"n_dim_{i}", 1, 512)

        # add layer and relu to list
        layer_list += [torch.nn.Linear(features, new_features), torch.nn.ReLU()]

        # set the new feature to be the current feature
        features = new_features

    # set the last layer - this one must map to the out dimension
    layer_list += [torch.nn.Linear(features, config.out_dim)]
    model = torch.nn.Sequential(*layer_list).to(device)

    # learning tools
    criterion = choice_loss(config)
    optimizer = choice_optim(config, model)

    # init score list
    score_list = []

    # train for specified number of epochs
    for epoch in range(config.num_max_epochs):
        _, _, _ = train_epoch(
            model,
            train_loader,
            optimizer,
            device,
            criterion
        )

        _, val_true_y, val_pred_y = eval_epoch(
            model,
            val_loader,
            device,
            criterion
        )

        # calc r2 score and append
        score = r2_score(y_true=val_true_y, y_pred=val_pred_y)
        score_list += [score]
        trial.report(score, epoch)

    # calculate objective
    # display(score_list)
    # idea: last x r2 scores (why not last one? for stability purposes)
    obj = np.array(score_list)[-stability_count:]
    return np.mean(obj)

## Tree advanced label

In [3]:
# setting global parameters
dataset_path = "../../datasets/"
label = "tree_advanced" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 70

In [ ]:
study_tree_advanced = optuna.create_study(study_name="Best tree advanced label config", directions=['maximize'])
study_tree_advanced.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-08 19:56:49,734] A new study created in memory with name: Best tree advanced label config


  0%|          | 0/70 [00:00<?, ?it/s]

[I 2023-12-08 19:57:58,943] Trial 0 finished with value: -939.4449011030545 and parameters: {'learning_rate': 2.4006605088264315e-05, 'batcher': False, 'n_layers': 1}. Best is trial 0 with value: -939.4449011030545.
[I 2023-12-08 19:59:19,023] Trial 1 finished with value: 0.9201144923544472 and parameters: {'learning_rate': 0.002104284711699982, 'batcher': False, 'n_layers': 4, 'n_dim_0': 219, 'n_dim_1': 401, 'n_dim_2': 355}. Best is trial 1 with value: 0.9201144923544472.
[I 2023-12-08 20:00:40,962] Trial 2 finished with value: -1004.7290627621171 and parameters: {'learning_rate': 3.064636911462103e-05, 'batcher': False, 'n_layers': 5, 'n_dim_0': 89, 'n_dim_1': 304, 'n_dim_2': 315, 'n_dim_3': 1}. Best is trial 1 with value: 0.9201144923544472.
[I 2023-12-08 20:02:05,508] Trial 3 finished with value: 0.8962070188960141 and parameters: {'learning_rate': 0.00029349149611142326, 'batcher': False, 'n_layers': 2, 'n_dim_0': 208}. Best is trial 1 with value: 0.9201144923544472.
[I 2023-12-08

In [ ]:
best_par_tree_advanced = study_tree_advanced.best_params
print(best_par_tree_advanced)

In [ ]:
study_tree_advanced.sampler.__class__.__name__

In [ ]:
df_tree_advanced = study_tree_advanced.trials_dataframe()
df_tree_advanced.to_csv("study_label-tree-advanced_baseline_normalized.csv")
display(df_tree_advanced)

## Tree label

In [ ]:
# setting global parameters
dataset_path = "../../datasets/"
label = "tree" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 50

In [ ]:
study_tree = optuna.create_study(study_name="Best tree label config", directions=['maximize'])
study_tree.optimize(objective, n_trials=n_trials, show_progress_bar=True)

In [ ]:
best_par_tree = study_tree.best_params
print(best_par_tree)

In [ ]:
study_tree.sampler.__class__.__name__

In [ ]:
df_tree = study_tree.trials_dataframe()
df_tree.to_csv("study_label-tree_baseline_normalized.csv")
display(df_tree)

## Default

In [ ]:
# setting global parameters
dataset_path = "../../datasets/"
label = "default" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 50

In [ ]:
study_default = optuna.create_study(study_name="Best default label config", directions=['maximize'])
study_default.optimize(objective, n_trials=n_trials, show_progress_bar=True)

In [ ]:
best_par_default = study_default.best_params
print(best_par_default)

In [ ]:
study_default.sampler.__class__.__name__

In [ ]:
df_default = study_default.trials_dataframe()
df_default.to_csv("study_label-default_baseline_normalized.csv")
display(df_default)